# Setup

In [ ]:
# Install dependencies
%pip install torch transformers huggingface_hub omegaconf
# Optinal packages for better user experience
%pip install tqdm ipywidgets nbconvert

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Import necessary libraries
import torch
import omegaconf
import collections
import typing
import os
from transformers import DPRQuestionEncoder, DPRContextEncoder, AutoTokenizer, AutoModel
from huggingface_hub import hf_hub_download

# Setup external services authentication
HF_TOKEN = os.getenv('HF_TOKEN')
print("esfera?")

esfera?


# Model Loading

In [ ]:
# Ensure that the necessary types are registered for safe deserialization
torch.serialization.add_safe_globals([
    omegaconf.dictconfig.ContainerMetadata,
    omegaconf.dictconfig.DictConfig,
    omegaconf.base.Metadata,
    omegaconf.nodes.AnyNode,
    omegaconf.listconfig.ListConfig,
    collections.defaultdict,
    typing.Any,
    dict,
    list,
    int])

# Load model from checkpoint
checkpoint_path = hf_hub_download(
    repo_id="NTU-NLP-sg/xCodeEval-nl-code-starencoder-ckpt-37",
    filename="dpr_biencoder.37.pt",
    repo_type="model",
    token=HF_TOKEN,
)
state_dict = torch.load(checkpoint_path, map_location="cpu")

# 2. Create custom DPR model class that matches the checkpoint structure
class StarEncoderDPR(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.question_model = AutoModel.from_pretrained("bigcode/starencoder", token=HF_TOKEN)
        self.ctx_model = AutoModel.from_pretrained("bigcode/starencoder", token=HF_TOKEN)
        
    def forward(self, *args, **kwargs):
        # Implement forward pass as needed
        pass

# Initialize encoders with StarEncoder architecture
# question_encoder = DPRQuestionEncoder.from_pretrained("bigcode/starencoder", token=HF_TOKEN)
# ctx_encoder = DPRContextEncoder.from_pretrained("bigcode/starencoder", token=HF_TOKEN)
question_encoder = DPRQuestionEncoder.from_pretrained("bigcode/starencoder", token=HF_TOKEN)
ctx_encoder = DPRContextEncoder.from_pretrained("bigcode/starencoder", token=HF_TOKEN)

# # Load fine-tuned weights
# def rename_keys(prefix):
#     return {k.replace(f"{prefix}.", ""): v for k, v in state_dict.items() if k.startswith(prefix)}
    
# question_encoder.load_state_dict(rename_keys("question_model"))
# ctx_encoder.load_state_dict(rename_keys("ctx_model"))

# Load state with careful key renaming
def load_weights(model, prefix):
    # Create mapping from checkpoint keys to model keys
    weights = {}
    for k, v in state_dict["model_dict"].items():
        if k.startswith(prefix):
            # Remove prefix and rename bert_model to just the base model
            new_key = k.replace(f"{prefix}.", "").replace("bert_model", "")
            weights[new_key] = v
    model.load_state_dict(weights, strict=True)


print(question_encoder)
print(ctx_encoder)
# load_weights(question_encoder, "question_model")
# load_weights(ctx_encoder, "ctx_model")


# # Check if CUDA is available and set device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

question_encoder = question_encoder.to(device).eval()
ctx_encoder = ctx_encoder.to(device).eval()
tokenizer = AutoTokenizer.from_pretrained("bigcode/starencoder")

You are using a model of type bert to instantiate a model of type dpr. This is not supported for all configurations of models and can yield errors.
Some weights of DPRQuestionEncoder were not initialized from the model checkpoint at bigcode/starencoder and are newly initialized: ['question_encoder.bert_model.embeddings.LayerNorm.bias', 'question_encoder.bert_model.embeddings.LayerNorm.weight', 'question_encoder.bert_model.embeddings.position_embeddings.weight', 'question_encoder.bert_model.embeddings.token_type_embeddings.weight', 'question_encoder.bert_model.embeddings.word_embeddings.weight', 'question_encoder.bert_model.encoder.layer.0.attention.output.LayerNorm.bias', 'question_encoder.bert_model.encoder.layer.0.attention.output.LayerNorm.weight', 'question_encoder.bert_model.encoder.layer.0.attention.output.dense.bias', 'question_encoder.bert_model.encoder.layer.0.attention.output.dense.weight', 'question_encoder.bert_model.encoder.layer.0.attention.self.key.bias', 'question_encod

DPRQuestionEncoder(
  (question_encoder): DPREncoder(
    (bert_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(49156, 768, padding_idx=49152)
        (position_embeddings): Embedding(1024, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0-11): 12 x BertLayer(
            (attention): BertAttention(
              (self): BertSdpaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_fe

RuntimeError: Error(s) in loading state_dict for DPRQuestionEncoder:
	Missing key(s) in state_dict: "question_encoder.bert_model.embeddings.word_embeddings.weight", "question_encoder.bert_model.embeddings.position_embeddings.weight", "question_encoder.bert_model.embeddings.token_type_embeddings.weight", "question_encoder.bert_model.embeddings.LayerNorm.weight", "question_encoder.bert_model.embeddings.LayerNorm.bias", "question_encoder.bert_model.encoder.layer.0.attention.self.query.weight", "question_encoder.bert_model.encoder.layer.0.attention.self.query.bias", "question_encoder.bert_model.encoder.layer.0.attention.self.key.weight", "question_encoder.bert_model.encoder.layer.0.attention.self.key.bias", "question_encoder.bert_model.encoder.layer.0.attention.self.value.weight", "question_encoder.bert_model.encoder.layer.0.attention.self.value.bias", "question_encoder.bert_model.encoder.layer.0.attention.output.dense.weight", "question_encoder.bert_model.encoder.layer.0.attention.output.